### UNETR++ (Synapse)

![Image of Yaktocat](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FbfoICR%2FbtrU1MMvL15%2FcwYgtGTptLML76AfvTRRM0%2Fimg.png)

- Input: 128 x 128 x 64 x 1
- Output: 128 x 128 x 64 x 14  
<br/>
- 구조:<br/> 
(1) input -> (encoder1) -> convblock  
(2) input -> (UnetrPPEncoder) -> (decoder5) -> (decoder4) -> (decoder3) -> (decoder2 with convblock) -> (out1) -> output
<br/>
<br/>
- 초록색 블록: UnetResBlock (Conv 3x3x3 -> Conv 3x3x3 -> skip connection) or (Conv 3x3x3 -> Conv 3x3x3 -> Conv 1x1x1 -> skip connection) <-- encoder1의 처음 ConvBlock  
- 분홍색 블록: TFblock (reshape -> positional encoding -> EPA -> skip connection -> reshape -> UnetResBlock -> Conv 1x1x1 -> skip connection )  
<br/>
- 인코더: UnetrPPEncoder (embedding<2,4,4> -> TFblock x3 -> (downsample->TFblock x3) ->...)  
<br/>
- 파란색 블록 + 분홍색 블록 : UnetrUpBlock ((transp_conv<2,2,2> -> skip connection -> TFblock x3) ->...)) // decoder5 ~ 3  
- 파란색 블록 + 초록색 블록 : UnetrUpBlock (transp_conv<2,4,4> -> skip connection -> UnetResBlock)  // decoder2  
<br/>
- 회색 블록: UnetOutBlock (Conv 1x1x1)  

학습 관련 파일

In [1]:
'''
1.unetr_pp/run/run_training.py -> start 
2.unetr_pp/run/default_configuration.py -> return trainer_class 
3-1.unetr_pp/training/network_training/unetr_pp_trainer_synapse.py -> trainer class
3-2.unetr_pp/training/network_training/Trainer_synapse.py -> trainer parent class
4.unetr_pp/network_architecture/synapse/unetr_pp_synapse.py -> network class (UNETR_PP)
4-1.unetr_pp/network_architecture/synapse/model_components.py -> encoder class (UnetrPPEncoder), decoder class (UnetrUpBlock)
4-2.unetr_pp/network_architecture/synapse/transformerblock.py -> Transformer Block, EPA class
4-3.unetr_pp/network_architecture/dynunet_block.py -> UnetResBlock, UnetOutBlock

(특이사항) encoder는 한 개의 모듈(UnetrPPEncoder)로 되어 있고, decoder는 여러 개의 모듈(UnetrUpBlock)로 되어 있음
'''

'\n1.unetr_pp/run/run_training.py -> start \n2.unetr_pp/run/default_configuration.py -> return trainer_class \n3-1.unetr_pp/training/network_training/unetr_pp_trainer_synapse.py -> trainer class\n3-2.unetr_pp/training/network_training/Trainer_synapse.py -> trainer parent class\n4.unetr_pp/network_architecture/synapse/unetr_pp_synapse.py -> network class (UNETR_PP)\n4-1.unetr_pp/network_architecture/synapse/model_components.py -> encoder class (UnetrPPEncoder), decoder class (UnetrUpBlock)\n4-2.unetr_pp/network_architecture/synapse/transformerblock.py -> Transformer Block, EPA class\n4-3.unetr_pp/network_architecture/dynunet_block.py -> UnetResBlock, UnetOutBlock\n\n(특이사항) encoder는 한 개의 모듈(UnetrPPEncoder)로 되어 있고, decoder는 여러 개의 모듈(UnetrUpBlock)로 되어 있음\n'

fold validation 설정 확인  
(fold 0만 이용한다고 함)

In [2]:
import pandas as pd

In [3]:
path='/media/ahnsunghyun/HDD/pytorch_hdd/unetr_plus_plus/DATASET/unetr_pp_raw/unetr_pp_raw_data/Task02_Synapse/Task002_Synapse/splits_final.pkl'
data = pd.read_pickle(path)
for i,dt in enumerate(data):
    print(i,dt)
    print()

0 OrderedDict([('train', array(['img0001', 'img0002', 'img0003', 'img0005', 'img0006', 'img0007',
       'img0008', 'img0010', 'img0021', 'img0022', 'img0024', 'img0025',
       'img0027', 'img0028', 'img0030', 'img0031', 'img0032', 'img0033',
       'img0034', 'img0035', 'img0036', 'img0038', 'img0039', 'img0040'],
      dtype='<U7')), ('val', array(['img0004', 'img0009', 'img0023', 'img0026', 'img0029', 'img0037'],
      dtype='<U7'))])

1 OrderedDict([('train', array(['img0002', 'img0003', 'img0004', 'img0005', 'img0006', 'img0009',
       'img0010', 'img0021', 'img0022', 'img0023', 'img0024', 'img0025',
       'img0026', 'img0027', 'img0028', 'img0029', 'img0030', 'img0033',
       'img0034', 'img0036', 'img0037', 'img0038', 'img0039', 'img0040'],
      dtype='<U7')), ('val', array(['img0001', 'img0007', 'img0008', 'img0031', 'img0032', 'img0035'],
      dtype='<U7'))])

2 OrderedDict([('train', array(['img0001', 'img0002', 'img0003', 'img0004', 'img0005', 'img0006',
       'img000

- 18 training and 12 validation cases:    
  
['img0005', 'img0006', 'img0007', 'img0009', 'img0010', 'img0021', 'img0023', 'img0024', 'img0026', 'img0027', 'img0028', 'img0030', 'img0031', 'img0033', 'img0034', 'img0037', 'img0039', 'img0040'],  
['img0001', 'img0002', 'img0003', 'img0004', 'img0008', 'img0022', 'img0025', 'img0029', 'img0032', 'img0035', 'img0036', 'img0038']

Synapse 데이터셋 분석

In [4]:
path2='/media/ahnsunghyun/HDD/pytorch_hdd/unetr_plus_plus/DATASET/unetr_pp_raw/unetr_pp_raw_data/Task02_Synapse/Task002_Synapse/unetr_pp_Plansv2.1_plans_3D.pkl'
plans = pd.read_pickle(path2)
print('num_modalities:',plans['num_modalities'], end='\n\n')
print('len(data):',len(plans['dataset_properties']['all_sizes']), end='\n\n')
print('original_sizes:',plans['original_sizes'], end='\n\n')
print('preprocessed_data_folder:',plans['preprocessed_data_folder'], end='\n\n')
print('all_classes:',plans['all_classes'], end='\n\n')
print('base_num_features:',plans['base_num_features'], end='\n\n')

num_modalities: 1

len(data): 30

original_sizes: [(147, 512, 512), (139, 512, 512), (198, 512, 512), (140, 512, 512), (117, 512, 512), (131, 512, 512), (163, 512, 512), (148, 512, 512), (149, 512, 512), (148, 512, 512), (143, 512, 512), (89, 512, 512), (96, 512, 512), (124, 512, 512), (85, 512, 512), (131, 512, 512), (88, 512, 512), (89, 512, 512), (100, 512, 512), (153, 512, 512), (93, 512, 512), (144, 512, 512), (104, 512, 512), (98, 512, 512), (94, 512, 512), (184, 512, 512), (99, 512, 512), (100, 512, 512), (90, 512, 512), (195, 512, 512)]

preprocessed_data_folder: ../DATASET/nnFormer_raw/nnFormer_raw_data/Task02_Synapse/Task002_Synapse

all_classes: [1, 10, 11, 12, 13, 2, 3, 4, 5, 6, 7, 8, 9]

base_num_features: 32



base_num_features: patch_embedding 이후 첫 번째 채널 수 (32)

배치 사이즈

In [5]:
stage = list(plans['plans_per_stage'].keys())[0]
stage_plans = plans['plans_per_stage'][stage]
batch_size = stage_plans['batch_size']
print(batch_size)

2


모델 구조  
(unetr_pp/training/network_training/unetr_pp_trainer_synapse.py,  Trainer_synapse.py 참조)

In [6]:
import torch
import torch.nn as nn
from pytorch_model_summary import summary
from unetr_pp.network_architecture.synapse.unetr_pp_synapse import UNETR_PP
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 

/home/ahnsunghyun/.local/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:115.)
  return torch._C._cuda_getDeviceCount() > 0


In [7]:
input_channels = plans['num_modalities']
num_classes=plans['num_classes'] + 1  # organ category + background
crop_size = [64, 128, 128]

In [8]:
print('===========================HyperParameters=============================')
print('input_channels',':',input_channels)
print('num_classes',':',num_classes)
print('crop_size',':',crop_size)
print('feature_size',':',16)
print('num_heads',':',4)
print('depths',':',[3, 3, 3, 3])
print('dims',':',[32, 64, 128, 256])
print('=========================================================================')

===========================HyperParameters=============================
input_channels : 1
num_classes : 14
crop_size : [64, 128, 128]
feature_size : 16
num_heads : 4
depths : [3, 3, 3, 3]
dims : [32, 64, 128, 256]


인코더1

In [9]:
from unetr_pp.network_architecture.dynunet_block import UnetResBlock

# 네트워크
encoder1 = UnetResBlock(
            spatial_dims=3,
            in_channels=1,
            out_channels=16,
            kernel_size=3,
            stride=1,
            norm_name="instance",
        )

# 테스트
input=torch.zeros(1, 1, 64, 128, 128) # Input Shape : [B, C, D, H, W]
output=encoder1(input) 
print('output shape:',output.shape) # Output Shape

output shape: torch.Size([1, 16, 64, 128, 128])


인코더 (수축경로)

In [10]:
from unetr_plus_plus.unetr_pp.network_architecture.synapse.model_components import UnetrPPEncoder

# 네트워크
unetr_pp_encoder = UnetrPPEncoder(dims=[32,64,128,256], depths=[3,3,3,3], num_heads=4)

# 테스트
input=torch.zeros(1, 1, 64, 128, 128) # Input Shape : [B, C, D, H, W]
output, hidden_states=unetr_pp_encoder(input) 
print('output shape:',output.shape) # Output Shape
print('hidden_states:')
for i,hd in enumerate(hidden_states):
    print(i,':',hd.shape)

output shape: torch.Size([1, 64, 256])
hidden_states:
0 : torch.Size([1, 32, 32, 32, 32])
1 : torch.Size([1, 64, 16, 16, 16])
2 : torch.Size([1, 128, 8, 8, 8])
3 : torch.Size([1, 64, 256])


proj_feat (인코더 출력 shape 변경)

In [11]:
# 네트워크
def proj_feat(x, hidden_size, feat_size):
    x = x.view(x.size(0), feat_size[0], feat_size[1], feat_size[2], hidden_size)
    x = x.permute(0, 4, 1, 2, 3).contiguous()
    return x

# 테스트
enc4 = hidden_states[3]
dec4 =proj_feat(enc4, 256,[4,4,4])
print('input:',enc4.shape)
print('output:',dec4.shape)

input: torch.Size([1, 64, 256])
output: torch.Size([1, 256, 4, 4, 4])


UnetrUpBlock (decoder5)

In [12]:
from unetr_pp.network_architecture.synapse.model_components import UnetrUpBlock

feature_size=16

# 네트워크
decoder5 = UnetrUpBlock(
            spatial_dims=3,
            in_channels=feature_size * 16,
            out_channels=feature_size * 8,
            kernel_size=3,
            upsample_kernel_size=2,
            norm_name="instance",
            out_size=8 * 8 * 8,
        )

# 테스트
dec4=torch.zeros(1,256,4,4,4)
enc3=hidden_states[2] # [1, 128, 8, 8, 8]
dec3=decoder5(dec4,enc3)
print('input(dec4,enc3):',dec4.shape, enc3.shape)
print('output:',dec3.shape)

input(dec4,enc3): torch.Size([1, 256, 4, 4, 4]) torch.Size([1, 128, 8, 8, 8])
output: torch.Size([1, 128, 8, 8, 8])


UnetrUpBlock (decoder2)

In [13]:
# 네트워크
decoder2 = UnetrUpBlock(
            spatial_dims=3,
            in_channels=feature_size * 2,
            out_channels=feature_size,
            kernel_size=3,
            upsample_kernel_size=(2, 4, 4),
            norm_name="instance",
            out_size=64 * 128 * 128,
            conv_decoder=True,
        )

# 테스트
dec1=torch.zeros(1,32,32,32,32) 
convBlock=torch.zeros(1,16,64,128,128)
out=decoder2(dec1,convBlock)
print('input(dec1,convBlock):',dec1.shape, convBlock.shape)
print('output:',out.shape)

input(dec1,convBlock): torch.Size([1, 32, 32, 32, 32]) torch.Size([1, 16, 64, 128, 128])
output: torch.Size([1, 16, 64, 128, 128])


out

In [14]:
from unetr_pp.network_architecture.dynunet_block import UnetOutBlock, UnetResBlock

out_channels=14

# 네트워크
out1 = UnetOutBlock(spatial_dims=3, in_channels=feature_size, out_channels=out_channels)

# 테스트
input=torch.zeros(1,16,64,128,128)
logits = out1(input)
print('input:',input.shape)
print('output:',logits.shape)

input: torch.Size([1, 16, 64, 128, 128])
output: torch.Size([1, 14, 64, 128, 128])


전체 모델

In [15]:
# 네트워크
network = UNETR_PP(in_channels=input_channels,
                   out_channels=num_classes,
                   img_size=crop_size,
                   feature_size=16,
                   num_heads=4,
                   depths=[3, 3, 3, 3],
                   dims=[32, 64, 128, 256],
                   do_ds=True,)
network=network.to(device) # Network

# 모델 요약 및 테스트
input=torch.zeros(1, 1, 64, 128, 128) # Input Shape : [B, C, D, H, W]
print(summary(network, input, show_input=True)) # Forwarding Shape
output=network(input) 
print('output shape:') # Output Shape
for i,out in enumerate(output):
    print(i,':',out.shape)

---------------------------------------------------------------------------------------------------
       Layer (type)                                    Input Shape         Param #     Tr. Param #
   UnetrPPEncoder-1                           [1, 1, 64, 128, 128]      27,387,232      27,387,232
     UnetResBlock-2                           [1, 1, 64, 128, 128]           7,360           7,360
     UnetrUpBlock-3           [1, 256, 4, 4, 4], [1, 128, 8, 8, 8]       3,509,848       3,509,848
     UnetrUpBlock-4         [1, 128, 8, 8, 8], [1, 64, 16, 16, 16]       2,377,624       2,377,624
     UnetrUpBlock-5       [1, 64, 16, 16, 16], [1, 32, 32, 32, 32]       9,638,968       9,638,968
     UnetrUpBlock-6     [1, 32, 32, 32, 32], [1, 16, 64, 128, 128]          30,208          30,208
     UnetOutBlock-7                          [1, 16, 64, 128, 128]             238             238
     UnetOutBlock-8                            [1, 32, 32, 32, 32]             462             462
     Unet

Input: [1, 1, 64, 128, 128] ->  
(UnetrPPEncoder-1) ->  [1, 256, 4, 4, 4] ->  
(UnetrUpBlock-3, decoder5) -> [1, 128, 8, 8, 8] ->  
(UnetrUpBlock-4, decoder4) ->  [1, 64, 16, 16, 16] ->  
(UnetrUpBlock-5, decoder3) ->  [1, 32, 32, 32, 32] ->  
(UnetrUpBlock-6, decoder2) ->  [1, 16, 64, 128, 128] ->  
(UnetOutBlock-7, out1) -> [1, 14, 64, 128, 128]  
Output: [1, 14, 64, 128, 128]

output이 3개인 이유는 unet++의 deep supervision을 이용하기 때문이다.  
즉 multiple output을 평균짓고 Loss를 구하겠다는 것인데, 이렇게 하면 multiple semantic level의 feature map을 획득할 수 있다고 한다.  
-> 더 좋은 성능을 보임

모델 전체 구조

In [16]:
network.parameters

<bound method Module.parameters of UNETR_PP(
  (unetr_pp_encoder): UnetrPPEncoder(
    (downsample_layers): ModuleList(
      (0): Sequential(
        (0): Convolution(
          (conv): Conv3d(1, 32, kernel_size=(2, 4, 4), stride=(2, 4, 4), bias=False)
        )
        (1): GroupNorm(1, 32, eps=1e-05, affine=True)
      )
      (1): Sequential(
        (0): Convolution(
          (conv): Conv3d(32, 64, kernel_size=(2, 2, 2), stride=(2, 2, 2), bias=False)
        )
        (1): GroupNorm(32, 64, eps=1e-05, affine=True)
      )
      (2): Sequential(
        (0): Convolution(
          (conv): Conv3d(64, 128, kernel_size=(2, 2, 2), stride=(2, 2, 2), bias=False)
        )
        (1): GroupNorm(64, 128, eps=1e-05, affine=True)
      )
      (3): Sequential(
        (0): Convolution(
          (conv): Conv3d(128, 256, kernel_size=(2, 2, 2), stride=(2, 2, 2), bias=False)
        )
        (1): GroupNorm(128, 256, eps=1e-05, affine=True)
      )
    )
    (stages): ModuleList(
      (0): S